<a href="https://colab.research.google.com/github/saideep3/Test/blob/master/Cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import keras
import os
import random
from keras.preprocessing import image
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model

from zipfile import ZipFile
file_name = "CIFAR10.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

data_folder = '/content/CIFAR10/cifar/TRAINSET'
data_folder_test = '/content/CIFAR10/cifar/TESTSET'

Done


In [0]:

train_data_split, test_data_split = 0.7, 0.3

full_class = [data[0] for data in os.walk(data_folder) if data[0]][1:]
#print(full_class)
full_class_test = [data[0] for data in os.walk(data_folder_test) if data[0]][1:]

def load_image(path):
    imag = image.load_img(path, target_size=(32, 32))
    x = image.img_to_array(imag)
    x = np.expand_dims(x, axis=0)
    return imag, x


train = []
test = []

for c, classs in enumerate(full_class):
    image_count = 0
    images = [os.path.join(dirp, file) for dirp, dirn, filenames
              in os.walk(classs) for file in filenames
              if os.path.splitext(file)[1].lower() in ['.jpg','.png','.jpeg','.bmp']]
  
    for img_path in images:
       
        imag, x = load_image(img_path)
        train.append({'x':np.array(x[0]), 'y':c})
        

num_classes = len(full_class)
num_classes_test = len(full_class_test)


tr_d = []
tr_y = []
tes_d = []
tes_y = []
count = 0
count1 = 0
for tr in train:
    tr_d.append(tr['x'])
    tr_y.append(tr['y'])
    #if(count<1000):
     #   count = count+1
    #else:
     #   break

train = None      

In [0]:
for c, classs in enumerate(full_class_test):
    image_count = 0
    images = [os.path.join(dirp, file) for dirp, dirn, filenames
              in os.walk(classs) for file in filenames
              if os.path.splitext(file)[1].lower() in ['.jpg','.png','.jpeg','.bmp']]
    
    for img_path in images:
        
        imag, x = load_image(img_path)
        test.append({'x':np.array(x[0]), 'y':c})

In [0]:


for te in test:
    tes_d.append(te['x'])
    tes_y.append(te['y'])
    #if (count1 < 1000):
     #   count1 = count1 + 1
    #else:
     #   break



test = None
full_class = None
train_X = np.array(tr_d)
train_Y = tr_y[:]
test_X = np.array(tes_d)
test_Y = tes_y[:]
X = None
Y = None
train_Y = keras.utils.to_categorical(train_Y, num_classes)
test_Y = keras.utils.to_categorical(test_Y, num_classes_test)
print(test_Y.shape)


(10000, 10)


In [0]:
vgg16_untr = keras.applications.VGG16(weights=None, include_top=True)
vgg16_untr.summary()

inpu = vgg16_untr.input
classification_layer = Dense(num_classes, activation='softmax')
outp = classification_layer(vgg16_untr.layers[-2].output)
model1 = Model(inpu, outp)

for l, layer in enumerate(model1.layers[:-1]):
    layer.trainable = False

for l, layer in enumerate(model1.layers[-1:]):
    layer.trainable = True

model1.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

model1.summary()

history1 = model1.fit(train_X, train_Y,
                             batch_size=128,
                             epochs=10, validation_data=(test_X, test_Y))
    #print(history2.history.keys())
print("Average training accuracy before pre-training: {}".format(np.mean([history1.history['acc'][-1],
                                                      history1.history['acc'][-1],
                                                      history1.history['acc'][-1]])))
print("Average testing accuracy before pre-training: {}".format(np.mean([history1.history['val_acc'][-1],
                                                     history1.history['val_acc'][-1],
                                                     history1.history['val_acc'][-1]])))


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

ValueError: ignored

In [0]:
no_of_training = []

vgg16 = keras.applications.VGG16(weights='imagenet', include_top=True)
vgg16.summary()

inp = vgg16.input
classification_layer = Dense(num_classes, activation='softmax')
out = classification_layer(vgg16.layers[-2].output)
model2 = Model(inp, out)

for l, layer in enumerate(model2.layers[:-1]):
    layer.trainable = False

for l, layer in enumerate(model2.layers[-1:]):
    layer.trainable = True

for i in range(1):

    model2.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                 metrics=['accuracy'])

    model2.summary()

    history2 = model2.fit(train_X, train_Y,
                             batch_size=128,
                             epochs=10, validation_data=(test_X, test_Y))
    #print(history2.history.keys())
    no_of_training.append(history2)


553467904/553467096 [==============================] - 6s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________________

KeyboardInterrupt: ignored

In [0]:

plt.subplot(2, 3, 1)
plt.plot(no_of_training[0].history['acc'])
plt.legend(['train1'], loc='lower right')

plt.subplot(2, 3, 2)
plt.plot(no_of_training[1].history['acc'])
plt.legend(['train2'], loc='lower right')

plt.subplot(2, 3, 3)
plt.plot(no_of_training[2].history['acc'])
plt.legend(['train3'], loc='lower right')

plt.subplot(2, 3, 4)
plt.plot(no_of_training[0].history['val_acc'])
plt.legend(['test1'], loc='lower right')

plt.subplot(2, 3, 5)
plt.plot(no_of_training[1].history['val_acc'])
plt.legend(['test2'], loc='lower right')

plt.subplot(2, 3, 6)
plt.plot(no_of_training[2].history['val_acc'])
plt.legend(['test3'], loc='lower right')

plt.tight_layout()

plt.show()

print("Average training accuracy After pre-training with imageNet: {}".format(np.mean([no_of_training[0].history['acc'][-1],
                                                      no_of_training[1].history['acc'][-1],
                                                      no_of_training[2].history['acc'][-1]])))
print("Average testing accuracy After pre-training with imageNet: {}".format(np.mean([no_of_training[0].history['val_acc'][-1],
                                                     no_of_training[1].history['val_acc'][-1],
                                                     no_of_training[2].history['val_acc'][-1]])))
